In [1]:
import pandas as pd
import numpy as np
from wmfdata import hive, charting
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import pct_str

## Read data

### Get data 2018

In [2]:
#2018 
t1_recipients =  pd.read_csv("../../data/raw/editors/2018/2018_grant_recipients.csv", sep=',', encoding = 'utf-8')

#ptp+ jury names
idc_jury_clean = pd.read_csv("../../data/processed/contest_api/indic_contests_jury_data.csv", sep=',', encoding = 'utf-8')

#ptp+ editor names
idc = pd.read_csv("../../data/processed/contest_api/indic_contests_data_clean.csv", sep=',', encoding = 'utf-8')

### Get data 2019

In [3]:
participants = pd.read_csv("../../data/processed/participants_list_received_clean/2019/participants.csv", sep=',', encoding = 'utf-8')

In [4]:
#editors_report_articles_per_month
erapm = pd.read_csv("../../data/raw/articles/2019/editors_report_articles_per_month.csv", sep=',', encoding = 'utf-8')

## Clean data

In [5]:
#idc_s = idc.loc[idc.str.startswith('project-tiger-2018-', na=False)]
idc_ptp18 = idc[idc['contest_code'].str.contains('project-tiger-2018-')]
idc_ptp18_editors = list(idc_ptp18['user_name'].unique())

In [6]:
len(idc_ptp18_editors)

289

In [7]:
idc_ptp18b = idc_ptp18.copy()
idc_ptp18b = idc_ptp18b[['article_name', 'user_name']]

In [8]:
idc_ptp18b = idc_ptp18b.rename(columns={'user_name': 'username',
                                        'article_name': 'page_title',
                                        })

In [9]:
erapm2018  = idc_ptp18b[['username', 'page_title']].groupby('username').count().sort_values(by='page_title', ascending=False).rename(columns={'page_title':'article_count'}).reset_index(inplace=True)
erapm2018['glow_editor_signup'] = True

In [11]:
len(erapm2018)

289

In [12]:
#create bool column for glow_editor_signup using Fountain data in idc_ptp18_editors list
t1_recipients["glow_editor_signup"]=np.where(t1_recipients["username"].isin(idc_ptp18_editors), 1,0)
t1_recipients['glow_editor_signup'] = t1_recipients['glow_editor_signup'].astype('bool')

#create column for recieved incentive
t1_recipients['incentive_recipient'] = True

In [13]:
t1_recipients.duplicated(subset='username').sum()

0

In [14]:
t1_recipients_clean = t1_recipients[['username', 'incentive_recipient', 'incentive_type', 'glow_editor_signup']]

### Method 1

### Method 2

In [21]:
participants_tally_2018 = pd.read_csv("../../data/processed/participants_list_received_clean/2018/PTP_participants_grantees_article_counts.csv", sep=',', encoding = 'utf-8')

In [25]:
cols1 = ['incentive_recipient', 'glow_editor_signup']
cols2 = ['incentive_type', 'article_count']

participants_tally_2018[cols1] = participants_tally_2018[cols1].fillna(False)
participants_tally_2018[cols2] = participants_tally_2018[cols2].fillna(0)

In [48]:
participants_tally_2018_clean = participants_tally_2018.copy()

In [49]:
participants_tally_2018_clean.to_csv("../../data/processed/participants_list_received_clean/2018/PTP_participants_grantees_article_counts_clean.csv", sep=',', encoding = 'utf-8', index=False)

## Clean data

### Method 1

In [ ]:
tp_filepath = "../../data/raw/editors/2019/tiger2_participants_report/total_participants.csv"
sr_filepath = "../../data/raw/editors/2019/tiger2_participants_report/support_recipients.csv"

#participants.to_csv(filepath, index=False, header=False)
contributors = pd.read_csv(tp_filepath)
support_recipients = pd.read_csv(sr_filepath)

In [ ]:
#TO DO - make all names lower case and care for double entries where incentive is diff

In [ ]:
#rename column for later merging
contributors = contributors.rename(columns={'Which incentive?':'incentive_type',
                                            'Recieved incentive?': 'incentive_recipient',
                                            'Username': 'username'
                                            })

support_recipients = support_recipients.rename(columns={'Recieved incentive?': 'received_incentive',
                                                        'Username': 'username',
                                                        'Incentive type':'incentive_type'
                                                       })

#clean up typos in the the incentive type column
contributors['incentive_type'] = contributors['incentive_type'].str.replace('Laptop, Internet' , 'laptop_internet')
contributors['incentive_type'] = contributors['incentive_type'].str.replace('Laptop,Internet' , 'laptop_internet')
support_recipients['incentive_type'] = support_recipients['incentive_type'].str.replace('Laptop,Internet' , 'laptop_internet')

#create a column for GLOW participation
contributors['glow_editor_signup'] = True

contributors['received_incentive'] = contributors['received_incentive'].replace(to_replace=['No', 'Yes'], value=[0, 1])

#handle column for 'Received incentive?'
contributors['received_incentive'] = contributors['received_incentive'].astype('bool')
support_recipients['received_incentive'] = support_recipients['received_incentive'].astype('bool')

In [ ]:
contributors['incentive_type'].unique()

In [ ]:
support_recipients['incentive_type'].unique()

In [ ]:
#identify editors not in editor list
e_p = list(contributors['username'])
e_r = list(support_recipients['username'])

support_recipients_missing = list(set(e_r)-set(e_p))
srta_r = support_recipients.loc[support_recipients['username'].isin(support_recipients_missing)]
srta = srta_r.copy()

#srta['glow_editor_signup'] = False

In [ ]:
participants = pd.concat([contributors, srta], sort=True).fillna(False)

### Method 2

In [32]:
replacements = {
    'incentive_type' : {
        'Laptop,Internet' : 'laptop_internet',
        'Laptop, Internet' : 'laptop_internet',
    }
}

participants.replace(replacements, inplace=True)

#lowercase values
participants['incentive_type'] = participants['incentive_type'].str.lower()

#handle column for 'Received incentive?'
participants = participants.rename(columns={'received_incentive': 'incentive_recipient'})


participants['incentive_recipient'] = participants['incentive_recipient'].replace(to_replace=['No', 'Yes'], value=[0, 1])
participants['incentive_recipient'] = participants['incentive_recipient'].astype('bool')

In [33]:
editor_tallies = erapm [['user', 'Grand Total']]

editor_tallies = editor_tallies.rename(columns={'user': 'username',
                                                'Grand Total': 'total_articles'
                                                       })

In [34]:
participants_tally_2019 = pd.merge(participants, editor_tallies, how='left', on='username').fillna(0)

In [38]:
participants_tally_2019.to_csv("../../data/processed/participants_list_received_clean/2019/participants_tally_2019.csv", sep=',', encoding = 'utf-8', index=False)

## Preparing to create table and load into hive

In [41]:
participants2018 = participants_tally_2018[['glow_editor_signup', 'incentive_recipient', 'incentive_type', 'username']]
participants2019 = participants_tally_2019[['glow_editor_signup', 'incentive_recipient', 'incentive_type', 'username']]

In [42]:
participants2018.to_csv("../../data/processed/participants_list_received_clean/final/participants2018.csv", sep=',', encoding = 'utf-8', index=False)

In [43]:
participants2019.to_csv("../../data/processed/participants_list_received_clean/final/participants2019.csv", sep=',', encoding = 'utf-8', index=False)

# Getting a list of participants into Hive

In [45]:
participants2019 = "../../data/processed/participants_list_received_clean/final/participants2019.csv"
hive.load_csv(
    participants2019,
    field_spec="glow_editor_signup boolean, incentive_recipient boolean, incentive_type string, username string",
    db_name="florez",
    table_name="glow_tiger_participants",
    sep=','
)

In [46]:
participants2018 = "../../data/processed/participants_list_received_clean/final/participants2018.csv"
hive.load_csv(
    participants2018,
    field_spec="glow_editor_signup boolean, incentive_recipient boolean, incentive_type string, username string",
    db_name="florez",
    table_name="glow_tiger_pilot_participants",
    sep=','
)